<h1 style="text-align: center">
<strong>UNALM: PRÁCTICA CALIFICADA 3</strong>
</h1>

**Integrantes:**
1. Pichardo Diestra, Oliverio
2. Ortiz Arone, Víctor Alejandro
3. Llamoca León, Israel

In [2]:
# Tu clave de API de Spoonacular
api_key = "1a60313c31a84c6d80a50e336bd18421"

In [17]:
import requests
# ID de la receta a la que quieres encontrar recetas similares
recipe_id = 47746

# Realizar una llamada a la API para obtener recetas similares
url = f"https://api.spoonacular.com/recipes/{recipe_id}/similar"
query_params = {"apiKey": api_key}
response = requests.get(url, params=query_params).json()

response

[{'id': 456541,
  'imageType': 'jpg',
  'title': 'Spiced Apples',
  'readyInMinutes': 45,
  'servings': 8,
  'sourceUrl': 'http://allrecipes.com/Recipe/Spiced-Apples/'},
 {'id': 403407,
  'imageType': 'jpg',
  'title': 'Glazed Spiced Apples',
  'readyInMinutes': 25,
  'servings': 10,
  'sourceUrl': 'http://www.tasteofhome.com/recipes/glazed-spiced-apples'},
 {'id': 48263,
  'imageType': 'jpg',
  'title': 'Chicken & Spiced Apples',
  'readyInMinutes': 35,
  'servings': 6,
  'sourceUrl': 'http://www.eatingwell.com/recipes/chicken_spiced_apples.html'},
 {'id': 313917,
  'imageType': 'jpeg',
  'title': 'Spiced Apples and Raisins',
  'readyInMinutes': 38,
  'servings': 4,
  'sourceUrl': 'http://www.foodnetwork.com/recipes/tyler-florence/spiced-apples-and-raisins-recipe.html'},
 {'id': 403413,
  'imageType': 'jpg',
  'title': 'Spiced Baked Apples',
  'readyInMinutes': 45,
  'servings': 2,
  'sourceUrl': 'http://www.tasteofhome.com/Recipes/spiced-baked-apples'},
 {'id': 586610,
  'imageType':

In [20]:
import pandas as pd
df=pd.DataFrame(response)

In [22]:
id=df["id"]
id

0    456541
1    403407
2     48263
3    313917
4    403413
5    586610
6    930836
7    151791
Name: id, dtype: int64

In [31]:

df_list = []

for ids in id:
    url = f"https://api.spoonacular.com/recipes/{ids}/nutritionWidget.json"
    query_params = {"apiKey": api_key}
    response = requests.get(url, params=query_params).json()
    df = pd.DataFrame(response["bad"])
    # agrega una columna "id" al DataFrame con el valor del identificador actual
    df["id"] = ids
    df_list.append(df)

result_df = pd.concat(df_list)
# resetea los índices en el DataFrame resultante
result_df.reset_index(drop=True, inplace=True)

In [32]:
result_df


,title,amount,indented,percentOfDailyNeeds,id
0,Calories,126,False,6.31,456541
1,Fat,0.32g,False,0.49,456541
2,Saturated Fat,0.05g,True,0.32,456541
3,Carbohydrates,33g,False,11.15,456541
4,Sugar,26g,True,29.52,456541
5,Cholesterol,0.0mg,False,0.00,456541
6,Sodium,1mg,False,0.08,456541
7,Calories,170,False,8.51,403407
8,Fat,0.32g,False,0.49,403407
9,Saturated Fat,0.06g,True,0.38,403407
